# Data Extraction - Comprehensive Azure AI Document Intelligence + Azure OpenAI GPT-4o with Vision

This sample demonstrates how to build a comprehensive process to extract structured data from any document using Azure AI Document Intelligence and Azure OpenAI's GPT-4o model with vision capabilities.

![Data Extraction](../../../images/extraction-comprehensive.png)

This is achieved by the following process:

- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the structure as Markdown.
- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes the specific extraction instruction for the type of document, the text content, and each document page as a base64 encoded image.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document to Markdown format using Azure AI Document Intelligence.
- Convert a document into a set of base64 encoded images for processing by GPT-4o.
- Use prompt engineering techniques to instruct GPT-4o to extract structured data from a type of document.
- Use the [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure) to extract structured data from the document page images using Azure OpenAI's GPT-4o model.
- Use the analysis result from Azure AI Document Intelligence to determine the confidence of the extracted structured output.
- Use the [logprobs](https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#request-body:~:text=False-,logprobs,-integer) parameter in an OpenAI request to determine the confidence of the extracted structured output.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **pdf2image** for converting a PDF file into a set of images per page.
- **azure-ai-documentintelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **openai** to interface with the Azure OpenAI chat completions API to generate structured extraction outputs using the GPT-4o model.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local modules are also used:

- **modules.app_settings** to access environment variables from the `.env` file.
- **modules.comparison** to compare the output of the extraction process with expected results.
- **modules.document_intelligence_confidence** to evaluate the confidence of the extraction process based on the extracted structured output and the analysis result from Azure AI Document Intelligence.
- **modules.document_processing_result** to store the results of the extraction process as a file.
- **modules.openai_confidence** to calculate the confidence of the classification process based on the `logprobs` response from the API request.
- **modules.vehicle_insurance_policy** to provide the expected structured output JSON schema for vehicle insurance policy documents.
- **modules.stopwatch** to measure the end-to-end execution time for the classification process.

In [1]:
import sys
sys.path.append('../../') # Import local modules

from IPython.display import display, Markdown
import os
import pandas as pd
from dotenv import dotenv_values
from pdf2image import convert_from_bytes
import base64
import io
from openai import AzureOpenAI
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, ContentFormat
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

from modules.app_settings import AppSettings
from modules.comparison import extraction_comparison
from modules.confidence import merge_confidence_values
from modules.document_intelligence_confidence import evaluate_confidence as di_evaluate_confidence
from modules.document_processing_result import DataExtractionResult
from modules.openai_confidence import evaluate_confidence as oai_evaluate_confidence
from modules.vehicle_insurance_policy import VehicleInsurancePolicy, VehicleInsuranceCostDetails, VehicleInsuranceRenewalDetails, VehicleInsurancePersonDetails, VehicleInsuranceVehicleDetails, VehicleInsuranceExcessDetails, VehicleInsurancePolicyEvaluator
from modules.stopwatch import Stopwatch

### Configure the Azure services

To use Azure AI Document Intelligence and Azure OpenAI, their SDKs are used to create client instances using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [2]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

openai_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')

openai_client = AzureOpenAI(
    azure_endpoint=settings.openai_endpoint,
    azure_ad_token_provider=openai_token_provider,
    api_version="2024-08-01-preview" # Requires the latest API version for structured outputs.
)

document_intelligence_client = DocumentIntelligenceClient(
    endpoint=settings.ai_services_endpoint,
    credential=credential
)

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on each page of the [Vehicle Insurance Policy](../../assets/VehicleInsurancePolicy.pdf).

The expected output has been defined by a human evaluating the document.

In [3]:
pdf_path = f"{working_dir}/samples/assets/"
pdf_file_name = "VehicleInsurancePolicy.pdf"
fname = f"{pdf_path}{pdf_file_name}"

expected = VehicleInsurancePolicy(
    policy_number='GB20246717948',
    cost=VehicleInsuranceCostDetails(
        annual_total=532.19,
        payable_by_date='2024-06-13'
    ),
    renewal=VehicleInsuranceRenewalDetails(
        renewal_notification_date='2025-05-12',
        last_date_to_renew='2025-05-26'
    ),
    effective_from='2024-06-03',
    effective_to='2025-06-02',
    last_date_to_cancel='2024-06-17',
    policyholder=VehicleInsurancePersonDetails(
        first_name='Joe',
        last_name='Bloggs',
        date_of_birth='1990-01-05',
        address='73 Regal Way, LEEDS, West Yorkshire, LS1 5AB',
        email_address='Joe.Bloggs@me.com',
        total_years_of_residence_in_uk=34,
        driving_license_number='BLOGGS901050JJ1AB'
    ),
    vehicle=VehicleInsuranceVehicleDetails(
        registration_number='VS24DMC',
        make='Hyundai',
        model='IONIQ 5 Premium 73 kWh RWD',
        year=2024,
        value=40000
    ),
    accident_excess=VehicleInsuranceExcessDetails(
        compulsory=250,
        voluntary=250,
        unapproved_repair_penalty=250
    ),
    fire_and_theft_excess=VehicleInsuranceExcessDetails(
        compulsory=250,
        voluntary=250,
        unapproved_repair_penalty=250
    ),
)

insurance_policy_evaluator = VehicleInsurancePolicyEvaluator(expected)

## Extract data from the document

The following code block executes the data extraction process using Azure OpenAI's GPT-4o model using vision capabilities.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use Azure AI Document Intelligence to analyze the structure of the document and convert it to Markdown format using the pre-built layout model.
3. Use pdf2image to convert the document's pages into images per page as base64 strings.
4. Using Azure OpenAI's GPT-4o model and its [Structured Outputs feature](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/structured-outputs?tabs=python-secure), extract a structured data transfer object (DTO) from the content of the images.

In [4]:
with Stopwatch() as di_stopwatch:
    with open(fname, "rb") as f:
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-layout",
            analyze_request=f,
            output_content_format=ContentFormat.MARKDOWN,
            content_type="application/pdf"
        )
    
    result: AnalyzeResult = poller.result()

markdown = result.content

# Prepare the user content for the OpenAI API including any specific details for processing this type of document, text, and the document page images.
user_content = []
user_content.append({
    "type": "text",
    "text": f"""Extract the data from this insurance policy. 
    - If a value is not present, provide null.
    - Some values must be inferred based on the rules defined in the policy.
    - Dates should be in the format YYYY-MM-DD."""
})
user_content.append({
    "type": "text",
    "text": markdown
})

with Stopwatch() as image_stopwatch:
    document_bytes = open(fname, "rb").read()
    page_images = convert_from_bytes(document_bytes)
    for page_image in page_images:
        byteIO = io.BytesIO()
        page_image.save(byteIO, format='PNG')
        base64_data = base64.b64encode(byteIO.getvalue()).decode('utf-8')
        
        user_content.append({
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_data}"
            }
        })

with Stopwatch() as oai_stopwatch:
    completion = openai_client.beta.chat.completions.parse(
        model=settings.gpt4o_model_deployment_name,
        messages=[
            {
                "role": "system",
                "content": "You are an AI assistant that extracts data from documents.",
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        response_format=VehicleInsurancePolicy,
        max_tokens=4096,
        temperature=0.1,
        top_p=0.1,
        logprobs=True # Enabled to determine the confidence of the response.
    )

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The accuracy of the structured data extraction comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the structured data extraction based on combining the confidence scores of the Azure AI Document Intelligence layout analysis and the log probability of the output generated by Azure OpenAI's GPT-4o model.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.
- The side-by-side comparison of the expected output and the output generated by Azure OpenAI's GPT-4o model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [5]:
# Displays the output of the Azure AI Document Intelligence pre-built layout analysis in Markdown format.
display(Markdown(markdown))

NexGen

<!-- PageHeader="Innovation drives progress" -->

Mr Joe Bloggs
73 Regal Way
LEEDS
West Yorkshire
LS1 5AB

3rd June 2024

Policy number

GB20246717948

Manage your account
NexGen.com/my-account

Hello Mr Bloggs,
Welcome to your new car insurance

Thanks for choosing us. Your new car insurance is effective from 3rd June 2024. Please check the summary table below
and let us know if there's anything incorrect.


<table>
<tr>
<th>Details</th>
<th>Option</th>
</tr>
<tr>
<td colspan="2">HYUNDAI IONIQ 5 PREMIUM 73 KWH RWD (2024) VS24DMC</td>
</tr>
<tr>
<td>Class Of Use</td>
<td>Business - class 1</td>
</tr>
<tr>
<td>Purchase Date</td>
<td>03/06/2024</td>
</tr>
<tr>
<td>Costs of insurance</td>
<td>£532.19 per year or £44.35 per month via Direct Debit</td>
</tr>
</table>


Your new documents
You'll find your new documents in the app or MyAccount.

· Schedule of insurance

· Statement of insurance

· Certificate of insurance

· Cover summary

We're here to help

If you have any questions about this, there are lots of ways you can get in touch. Please visit the Help Centre on our
website for details.

Thanks,
Jim Taylor
Operations Director

<!-- PageNumber="Insurance Policy | Page 1" -->
<!-- PageBreak -->

NexGen

Document issued: 3rd June
2024
Reason for issue: New contract
Policy number: GB20246717948


# Your statement of insurance

Please read and check the information shown in this document carefully. It represents the answers given during the initial quotation
process, and any changes that have been made to the policy since then. It forms part of your contract for motor insurance.

If any of the details are incorrect or have changed, please contact us immediately.

You can do this:

· Through MyAccount

· By calling 0123 321 1234

We'll tell you if this changes your premium, terms or insurer.

Under the Consumer Insurance (Disclosure and Representations) Act 2012, you have a duty to take reasonable care to answer all
questions fully and accurately. If you volunteer information over and above that requested, you must do so honestly and carefully. If
you don't answer all questions fully and accurately, it could invalidate your insurance cover and result in all, or part of a claim not
being paid.


<table>
<tr>
<td colspan="2">Policyholder details</td>
</tr>
<tr>
<td>Policyholder</td>
<td>Mr Joe Bloggs</td>
</tr>
<tr>
<td>Email address</td>
<td>Joe.Bloggs@me.com</td>
</tr>
<tr>
<td>Have any drivers on this policy ever had an insurance policy declined, cancelled, voided or had any special terms imposed?</td>
<td>No</td>
</tr>
<tr>
<td>Total vehicles in household</td>
<td>1</td>
</tr>
<tr>
<td>Total number of drivers in household</td>
<td>1</td>
</tr>
<tr>
<td>Your address</td>
<td>73 Regal Way, LEEDS, West Yorkshire, LS1 5AB</td>
</tr>
<tr>
<td>Years at address</td>
<td>4</td>
</tr>
<tr>
<td>Homeowner</td>
<td>Yes</td>
</tr>
<tr>
<td>Children under 16</td>
<td>No</td>
</tr>
<tr>
<td>Vehicle key at address overnight</td>
<td>Yes</td>
</tr>
<tr>
<td>Kept overnight</td>
<td>Locked Garage</td>
</tr>
</table>


<!-- PageNumber="Insurance Policy | Page 2" -->
<!-- PageBreak -->


<table>
<tr>
<td colspan="2">Insured drivers</td>
</tr>
<tr>
<td>Name</td>
<td>Joe Bloggs</td>
</tr>
<tr>
<td>Date of birth</td>
<td>05/01/1990</td>
</tr>
<tr>
<td>Marital status</td>
<td>Married-Common Law</td>
</tr>
<tr>
<td>Permanent resident in the UK since</td>
<td>Since birth</td>
</tr>
<tr>
<td>Employment status</td>
<td>Employed</td>
</tr>
<tr>
<td>Primary occupation</td>
<td>Software Engineer</td>
</tr>
<tr>
<td>Industry</td>
<td>Computers - Software</td>
</tr>
<tr>
<td>Driving licence number</td>
<td>BLOGGS901050JJ1AB</td>
</tr>
<tr>
<td>Medical conditions reportable to the DVLA/DVANI</td>
<td>No</td>
</tr>
<tr>
<td>Use of other vehicle</td>
<td>No</td>
</tr>
<tr>
<td>Unspent non-motoring convictions</td>
<td>No</td>
</tr>
</table>


## Your policy details


<table>
<tr>
<th>Cover type</th>
<th>Comprehensive</th>
</tr>
<tr>
<td>Class of use</td>
<td>Business - class 1</td>
</tr>
<tr>
<td>Your no claims discount</td>
<td>You told us you have 9 or more years of no claims discount, but we haven't confirmed this with the previous insurer. If you had a non-recoverable claim in this policy period, we'll adjust your no claims discount at renewal.</td>
</tr>
<tr>
<td>How no claims discount was earned</td>
<td>Private car discount</td>
</tr>
</table>


## Incident history

For all drivers named in this policy, we need to know of any incident, claim or damage involving any motor vehicle in the past five
years, including windscreen damage. This applies whether a claim was made, and regardless of blame.

None disclosed


## Conviction history

For all drivers named in this policy, we need to know of any driving related convictions, endorsements, fixed penalties,
disqualifications or bans in the past five years.


<table>
<tr>
<th>Driver</th>
<th>Date</th>
<th>Type</th>
<th>Licence points</th>
<th>Disqualification</th>
</tr>
<tr>
<td>Joe Bloggs</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>-</td>
</tr>
</table>


<!-- PageFooter="Insurance Policy | Page 3" -->
<!-- PageBreak -->


<table>
<tr>
<td>Insured vehicle details</td>
<td></td>
</tr>
<tr>
<td>Registration number</td>
<td>VS24DMC</td>
</tr>
<tr>
<td>Annual mileage</td>
<td>5,000</td>
</tr>
<tr>
<td>Make &amp; model</td>
<td>Hyundai IONIQ 5 Premium 73 kWh RWD</td>
</tr>
<tr>
<td>Body type</td>
<td>HATCHBACK</td>
</tr>
<tr>
<td>Year registered</td>
<td>2024</td>
</tr>
<tr>
<td>Transmission</td>
<td>Automatic</td>
</tr>
<tr>
<td>Engine size</td>
<td>0</td>
</tr>
<tr>
<td>Number of seats</td>
<td>5</td>
</tr>
<tr>
<td>Number of doors</td>
<td>5</td>
</tr>
<tr>
<td>Date purchased</td>
<td>05/2024</td>
</tr>
<tr>
<td>Imported</td>
<td>No</td>
</tr>
<tr>
<td>Left hand drive</td>
<td>No</td>
</tr>
<tr>
<td>Declared value</td>
<td>£40,000.00</td>
</tr>
<tr>
<td>Registered keeper &amp; legal owner</td>
<td>Yes</td>
</tr>
<tr>
<td>Security devices or immobiliser</td>
<td>Factory fitted Alarm + Immobiliser</td>
</tr>
<tr>
<td>Tracker fitted</td>
<td>No</td>
</tr>
<tr>
<td>Modifications</td>
<td>No</td>
</tr>
</table>


What's a modification?

A modification is any alteration to your car from the manufacturer's standard specification. This includes, but isn't limited to:

· Changes to the bodywork, such as spoilers or body kit

· Changes to suspension or brakes

· Alloy wheels

· Audio/entertainment system

· Changes affecting performance, such as to the engine management system or exhaust system.

You must also update your Policy if you intend to alter or modify your Car/s from the manufacturer's standard specification
If you don't tell us about a modification, we may cancel your Policy from its start date, apply additional premium or add new terms to
your Policy. If you make a claim your insurer may reject the claim or only provide partial payment for it.
We may need to check your vehicle for modifications and accessories at the time of your claim.

Please note: optional extras fitted at the point of manufacture aren't classed as modifications.

<!-- PageNumber="Insurance Policy | Page 4" -->
<!-- PageBreak -->

NexGen

Cover effective dates:

Reason for issue:

Policy number:

Insurer:
All mentions to "you" or "your" refer to:

10.41 on 3rd June 2024 to
2nd June 2025

New contract
GB20246717948
NexGen
Mr Joe Bloggs


# Schedule of insurance

You have comprehensive insurance. This means sections 1 to 8 of the policy apply.


## Excesses

Your excess is the part of a claim you must pay, even if the damage or loss isn't your fault. It also applies if a named driver was in
charge of the car.


<table>
<tr>
<th>Accidental damage</th>
<th>Compulsory</th>
<th>Voluntary</th>
<th>Total</th>
</tr>
<tr>
<td>Joe Bloggs</td>
<td>£250</td>
<td>£250</td>
<td>£500</td>
</tr>
<tr>
<td>Fire and theft</td>
<td>Compulsory</td>
<td>Voluntary</td>
<td>Total</td>
</tr>
<tr>
<td>All drivers</td>
<td>£250</td>
<td>£250</td>
<td>£500</td>
</tr>
</table>


<table>
<tr>
<th>Windscreen/glass</th>
<th>Repair Replacement</th>
</tr>
<tr>
<td>All drivers</td>
<td>£25 £110</td>
</tr>
</table>


These excesses are based on you using one of our approved repairers.
You can use your own repairer for an accidental damage or fire and theft claim but you'll still have to pay the excess shown above,
plus an additional excess of £250.

<!-- PageNumber="Insurance Policy | Page 5" -->
<!-- PageBreak -->


# Certificate of motor insurance

1\. Registration mark of vehicle: VS24DMC

Courtesy (or replacement) vehicles ("Replacement Vehicle") are covered on your policy when supplied to you or any named
driver(s) (set out in number 5 of this certificate) by your Insurer's nominated suppliers either:

a) whilst your vehicle is being repaired;
b) at your Insurer's request.

An exception to this is if the nominated supplier of the courtesy or replacement vehicle provides their own insurance cover
to you. This will be set out in your agreement with them.

If your Replacement Vehicle is covered by your policy, but your policy expiries or is cancelled, we'll automatically extend
this insurance, for the Replacement Vehicle only, at no cost to you, until you return it to the supplier on the date agreed
("Extended Cover"). During the Extended Cover (i) you'll only be insured to drive the Replacement Vehicle and no other
vehicles (including those set out in number 5 of this certificate); and (ii) if there's a claim involving the Replacement Vehicle,
you'll need to pay any excess set out in your policy.

2\.
. Name of policyholder: Mr Joe Bloggs

3\. Effective date of the commencement of insurance for the purpose of the relevant law
Your insurance starts at
10:41 on 3rd June 2024

4\. Date of expiry of insurance
Your insurance ends at
23:59 on 2nd June 2025

5\. Persons or classes of persons entitled to drive:

· Mr Joe Bloggs

Cover is provided only if the person driving:

· Holds a valid licence

· Isn't disqualified from holding or obtaining a valid licence.

Driving other cars:

As the policyholder, you're covered to drive someone else's car if:

· You have the owner's permission

· It has valid insurance

· It's not owned by you or your partner or business partner/employer

· It's not under a hire purchase, self-drive hire or lease agreement of any kind.

Named drivers do not have this cover and other conditions apply - see your policy details.


## 6. Limitations as to use

What's covered:

· Social, domestic and pleasure use

· Travelling to and from one or more permanent place(s) of business or study

· Use for your private business

What's not covered:

· Commercial travel, soliciting for orders, any use connected with the motor trade and hire or reward

· For racing on a public highway

· Use at any event where your vehicle is driven:

o On a motor-racing track, including de-restricted toll roads such as the Nürburgring
☐

o On a prepared course
☐

o At any off-road event, such as a 4x4 event
☐

o At an airfield
☐

<!-- PageNumber="Insurance Policy | Page 6" -->
<!-- PageBreak -->

NexGen

<!-- PageHeader="Innovation drives progress" -->


# LEGAL STATEMENTS


## 1. POLICYHOLDER OF A CAR POLICY

A policyholder of a car policy enters two separate contracts when taking out a policy through us.

(i)
The first contract is between the policyholder and the insurer:

a. The insurer's name is shown on the policyholder's current certificate of motor insurance. This can be
found on the policyholder's certificate of motor insurance in the app and MyAccount

b. The insurer is the company providing the policyholder's motor insurance

c. It is the policyholder's responsibility to be aware of these terms and conditions and to make sure that
named drivers are also aware of them

d. The policyholder is the only individual able to cancel the car policy (as set out in more detail in
'Cancellations')

e. The policyholder can make claims under the car policy on their own behalf and on behalf of any named
drivers in accordance with the terms of this policy

f.
This contract does not give rise to any rights under the Contracts (Rights of Third Parties) Act 1999 to
enforce any term of the contract

g. This contract will be governed by and interpreted in accordance with English law

h. The insurer will communicate in English throughout the course of this contract.

(ii)
The second contract is between the policyholder and NexGen ("we/us/our"):

a. We are an insurance broker, and we arrange and administer the policyholder's car policy on behalf of the
insurer

b. We are the policyholder's first point of contact

c. This contract does not give rise to any rights under the Contracts (Rights of Third Parties) Act 1999 to
enforce any term of the contract

d. This contract will be governed by and interpreted in accordance with English law

e. We will communicate in English throughout the course of this contract.

The policyholder of a car policy is covered by the insurer for the period of cover when the policyholder:

· Agrees to the terms and conditions offered; and

· Pays, or has offered to pay, the costs of insurance for the car policy.

The policyholder is required to take reasonable care not to make a misrepresentation when providing information to the
insurer and/or us.


## 2. PAYMENT

The costs of insurance may be paid:

(i)
In full by debit or credit card; or

(ii)
By instalments as agreed in a credit agreement (subject to eligibility)

If the costs of insurance are paid in one single payment, that payment may be made by a policyholder or a third party.

A credit agreement can only be with a policyholder. The instalments may be paid by anyone.


## 3. LEGAL OBLIGATIONS

It's an offence under the road traffic act to make a false statement or to withhold material information to get motor
insurance.

Under the Consumer Insurance (Disclosure and Representation) Act 2012, when you apply for insurance, you have a duty
to take reasonable care to answer all questions as fully and as accurately as possible.

<!-- PageFooter="Insurance Policy | Page 7" -->
<!-- PageBreak -->

If you do not take reasonable care to answer all questions fully and accurately or if you deliberately make a false statement
or withhold material information, there could be serious consequences. In certain circumstances your insurer, or we acting
on your insurer's behalf, have the right to treat your policy as if it never existed, without giving you notice or refunding your
premium. This will only apply:

. If you make a careless misrepresentation and your insurer would not have offered you insurance had it known the
true facts; or

· if you make a misrepresentation which is deliberate and/or reckless.

If the policy is treated as if it never existed, any claims made before or during this time will be declined and not paid and
you may have to make a payment to anybody involved in an incident, or to your insurer if it is obliged by law to make a
payment to anybody involved in an incident.

If the details on your documents are wrong, please contact our customer services team as soon as possible. Their details
are at the front of this document.

If you want to make a change to your policy, please contact our customer service team. Customers with online policies
should tell us about the changes on the app or MyAccount website.


### SECTION 1


#### DAMAGE TO YOUR CAR/S (EXCEPT THAT CAUSED BY FIRE OR THEFT)

You are covered for accident, vandalism and malicious damage

If your car/s is damaged or lost because of an accident, vandalism or malicious damage there are four ways your policy
can help you get back on the road again. Your insurer will do one of the following:

· Pay for any necessary repairs (including damage to charging cables and batteries for electric and hybrid vehicles)

. Replace your car

· Repair the damage

· Pay the market value of your car immediately before the loss.

Accessories are also covered while they are in, or on, your car or in your private garage.


#### SECTION 2


##### DAMAGE OR LOSS CAUSED BY FIRE OR THEFT

You are covered for fire, theft, attempted theft or lightning damage to your car/s

If your car is damaged or lost because of theft, attempted theft, fire or lightning there are four ways your policy can help
you get back on the road again. Your insurer will do one of the following:

· Pay for any necessary repairs

· Replace your car

· Repair the damage

· Pay the market value of your car immediately before the loss.

Accessories are also covered while they are in, or on, your car or in your private garage.

WHAT ISN'T COVERED UNDER SECTIONS 1 AND 2

You are not covered for:

· Mechanical, electrical, electronic or computer failures (including failures caused by a computer virus or cyber-
attack) or breakdowns or breakages

· Loss, damage or corruption of data caused by failure to install and/or accept vehicle manufacturer software
updates

. The excesses shown on your schedule of insurance - you will have to pay these if you make a claim

. Loss of use of your car (if you are out of pocket because you can't use your car, including the cost of hiring
another vehicle)

<!-- PageNumber="Insurance Policy | Page 8" -->
<!-- PageBreak -->

· Wear and tear, deterioration, depreciation, or any loss or damage that happens gradually

· Failures, breakdowns or breakage of mechanical, electrical, electronic or computer equipment

· Damage to tyres caused by braking, punctures, cuts or bursts

· Loss of value following repair

· Theft of or damage, if the car keys were left in or on the car or if the car is left unattended with the engine running

· Replacement of locks, if the car keys were left in or on the car or if the car is left unattended with the engine
running.

· Loss or damage if someone claiming to be a buyer or agent takes possession of your car deceitfully

· Your car being repossessed by its rightful owner or having to pay compensation to the owner

· Any amount greater than the manufacturer's last list price for replacing any part or accessories lost or damaged

· Repairs or replacements unrelated to your claim that improve the condition of your car

· Loss or damage because of your car being driven or used without your permission by a member of your family or
household unless the incident is reported to the police, and you send us the crime reference number

· Loss or damage caused by an inappropriate type or grade of fuel being used

· Loss or damage because of malicious damage or vandalism, where the police refuse to issue a crime reference
number. Please note that having a crime reference number doesn't guarantee we will settle a claim

· Any additional damage resulting from your car being moved by anyone insured under your policy after an
accident, fire or theft

· Loss or damage resulting from the legal confiscation of your car by HM Revenue and Customs, the police, a local
authority or any other government authority.


## HOW YOUR CLAIMS ARE SETTLED FOR SECTIONS 1 AND 2

How the insurer will deal with your claim for accident, vandalism, malicious damage, theft, attempted theft, fire or
lightning

If your car is damaged, your insurer will arrange the transportation of your car to the nearest suitable nominated repairer
or a place of storage. Where appropriate they will also return it after repair to the address shown on your schedule.
Alternatively, they will cover the reasonable cost of doing this.

If your car is stolen and not recovered then as soon as a total loss settlement is agreed and paid by your insurer, your
insurer will thereby take ownership of your car (where it is entitled to do so) and should your car be subsequently
recovered, any salvage shall become your insurer's property.

Replacement of locks and stolen keys

Provided it can be established to your insurer's reasonable satisfaction that the identity or garaging address of your car is
known to any person who may have stolen or found your keys and the value of your claim does not exceed the market
value of your car, your insurer will pay up to a maximum of £500 after deducting any excess, towards the cost of replacing:

. The door locks and/or boot lock

· The ignition/steering lock

. The lock transmitter and central locking interface.

You are not covered for stolen keys if they were left in or on your car while it was unattended or unoccupied.


### Audio visual equipment

Your insurer will cover the loss or damage to in-car audio, television, DVD, phone, games-console, dashboard camera,
electronic navigation or radar detection equipment permanently fitted to your car. This cover is unlimited if the equipment
was fitted by the manufacturer and was part of the specification of your car when first registered. If the equipment wasn't
originally part of your car, the most your insurer will pay is £300. Your insurer will settle a claim for audio visual equipment
by repairing it, replacing it with a similar piece of equipment or providing a cash payment.


### Costs you must pay

The total excess that applies to your claim. Your compulsory and voluntary excesses are based on you using your
insurer's nominated repairer. If you use a different repairer, there will be an additional excess to pay as shown in your
schedule of insurance.

<!-- PageNumber="Insurance Policy | Page 9" -->
<!-- PageBreak -->


### Costs you may have to pay

. If your insurer accepts your claim, and finds your details or circumstances have changed since you took your
policy out, you may have to pay any additional costs and associated fees (see General Conditions for more
details)

. If your claim is settled on a total loss basis and you pay by instalments under a loan arrangement with us, we may
take all outstanding payments from the claims settlement or ask you to pay the outstanding amount (see total loss
section for more details)

. If your insurer doesn't accept your claim, you may have to pay any costs already incurred. These may include (but
are not limited to) engineers' fees, vehicle recovery and storage charges.

Total loss - if your car can't be repaired

If your car can't be repaired or your insurer deems your car to be unsafe or the cost of repair to be uneconomical, your
car will be declared a total loss (sometimes called a 'write-off'). If your car/s are a total loss, your insurer may put it in
storage until your claim is settled. As soon as a total loss settlement is agreed and paid by your insurer, your insurer will
thereby take possession and ownership of your car (where it is entitled to do so) and any salvage shall become your
insurer's property.

If you are paying for your policy by instalments under a loan arrangement with us and your insurer settles a total loss claim
under these sections, your consumer credit agreement with us may entitle us to do one of the following:

. Take the outstanding amount due for your consumer credit agreement out of the claims settlement

. Require you to pay the outstanding amount due for the car in question.

If your car is declared a total loss, and you have already paid the premium in full, no refund will be made for the car in
question, even if the cover for the car is later cancelled. This may not apply if your insurer is able to recover all losses from
a third party. In this case insurers may sometimes refund the premium paid and, if they do, we will pass that refund on to
you.

If your claim is settled on a total loss basis and you don't replace your car within 30 days of being issued the settlement
payment, we will cancel your policy. The provisions above relating to loan agreements and refunds will still apply.


## SECTION 3


### LEGAL RESPONSIBILITY TO OTHERS (THIRD PARTIES)

What's covered

Your insurer will pay all sums you are liable for in respect of (1) death or injury to other people or (2) (up to £20,000,000
plus up to £5,000,000 for costs and expenses) damage to someone else's property caused by or arising from the use of
your car/s (or any other vehicle your policy covers you to drive - see your certificate of motor insurance).

This cover also applies to accidents involving a trailer, caravan or broken-down vehicle being towed (as long as you hold
the correct entitlement on your driving licence to do so).

Legal costs

Following a claim covered by this policy and if your insurer agrees it's in their interest to do so, which is entirely their
decision, they will pay reasonable legal costs and expenses for:

· Solicitors' fees for representing anyone insured at a coroner's inquest, fatal accident inquiry or court

· Reasonable legal services, which they will arrange, to defend a charge of manslaughter or causing death by
dangerous or reckless driving

. Any other legal costs and expenses if agreed in writing beforehand.

You must get your insurer's consent in writing before incurring these sorts of fees and costs.

What's not covered under section 3

. Anyone who has any other insurance covering the same liability

<!-- PageFooter="Insurance Policy | Page 10" -->
<!-- PageBreak -->

· Liability for the death of or injury to anyone while they are working with, or for, the driver of the car except to the
extent that the driver is required to be covered for such a liability by the road traffic act

· Any damage to personal property owned by the person driving your car at the time of the incident

. Loss of, or damage to, any trailer, caravan or vehicle (or their contents) while being towed by or attached to any
vehicle covered by this section

. Loss or damage to property of more than £20,000,000 for any one incident or series of incidents and costs and
expenses over £5,000,000.


## SECTION 4


### PERSONAL ACCIDENT

Your insurer will pay £5,000 if you or your partner are accidentally injured while travelling in or while getting into, or
getting out of, your car/s and within 90 days if this injury is the sole cause of:

· Death

· Permanent loss of sight in one or both eyes

. Total physical loss of a limb at or above the ankle or wrist. Your insurer will pay the injured person or their legal
representative.

What's not covered under section 4

· Death or injury resulting from suicide or attempted suicide

· Death or injury to anyone not wearing a seat belt when required by law

· Death or injury because the driver was unfit to drive because of alcohol, drugs or other substances, whether
prescribed or otherwise

· Any disablement, whether temporary, permanent, partial or total, except those listed above

· Injury caused by a pre-existing disease or physical weakness

· Anything excluded by the general exceptions listed later in this document.


# SECTION 5


## MEDICAL EXPENSES

If you, or anyone in your car, is injured in an accident, your insurer will pay medical expenses of up to £500 for each
injured person.


# SECTION 6


## PERSONAL BELONGINGS

What's covered

If you have comprehensive cover and you are making a claim under sections 1 or 2 of this policy, your insurer will pay up
to £300 for any one claim for personal belongings in your car, to you or the owner of the items.

What's not covered under section 6

· Money, stamps, jewellery, watches, tickets, credit or debit cards, vouchers, documents or securities (such as
share and premium bond certificates)

· Laptops, mobile phones, tablet computers or electronic navigational equipment

· Goods, samples or tools carried in connection with any trade or business

· Property insured under any other insurance policy

· Theft or attempted theft of personal belongings not kept out of sight in the glove compartment or locked boot

· Theft or attempted theft if the car was left unlocked while unoccupied or unattended

· Anything excluded by the general exceptions listed later in this document.

Your insurer may require documentary evidence to confirm your claim and/or may ask to see the damaged item.

<!-- PageFooter="Insurance Policy | Page 11" -->
<!-- PageBreak -->


# SECTION 7


## WINDSCREEN DAMAGE


### What's covered

If you have comprehensive cover your insurer will pay to replace, or repair broken glass in the windscreen or windows of
your car and repair any scratching to the bodywork caused by the broken glass.

Making a claim under this section won't affect your no claims discount, if you are not also claiming for any other loss or
damage to your car.

Your insurer's nominated repairer may use parts or accessories that aren't made or supplied by your car's manufacturer
but are of an equivalent type and quality to those being replaced.

What's not covered under section 7

· Any other glass forming part of your car including sunroofs, panoramic roofs or panoramic sunroofs, where the
roof glass is a separate unit to the windscreen glass

· Any windscreens or windows not made of glass

· Replacement of the hood/roof structure of a convertible or cabriolet car

. A repair or replacement cost that's more than the market value of your car at the time of loss (less any excess).

Additional charges or limited cover may apply if the repair isn't carried out by your insurer's nominated repairer.

You must pay an excess for windscreen, windows and glass repairs or replacement - see your schedule of insurance and
certificate of motor insurance for more details.

Any repairs to a windscreen will only be guaranteed for 30 days.


## SECTION 8


### UNINSURED DRIVER PROMISE

Your insurer promises that if you are involved in an accident that isn't your fault and the driver of the vehicle that hits you
doesn't have motor insurance:

. You won't lose your no claims discount

· You won't have to pay any excess/es.

To benefit from this promise, you must send us the make, model and registration number of the vehicle that caused
damage to your car and, when possible, tell us the other driver's name and address.

When you make a claim, you may initially have to pay your excess/es. If investigations are still taking place when your
renewal is due, you may lose your no claims discount temporarily, as explained above. Once your insurer has confirmed
the accident was the fault of an identified uninsured driver, your insurer will refund your excess, restore your no claims
discount and refund any extra premium you have paid.


## PAYMENTS

The total price of your insurance is shown in your documents and includes insurance premium tax. For legal purposes, we
must tell you that in future other taxes or costs may apply that are not paid for or imposed by us. However, at present, we
are not aware of any other taxes or costs payable.

If the policy is paid annually, the costs of insurance will be due within 10 days after the effective date of the
commencement for the policy. After this date, the policy will be cancelled.

<!-- PageFooter="Insurance Policy | Page 12" -->
<!-- PageBreak -->

If the policy is paid monthly, a monthly direct debit for continuous payment is required within 10 days after the effective
date of the commencement for the policy. We will use the direct debit details to continue to collect the costs of insurance
as agreed previously.

You must contact us as soon as possible to pay the costs of insurance.


# RENEWING YOUR POLICY

This section contains important information about renewing your policy.


## RENEWAL PERIOD

21 days before your policy ends, we will send a renewal notice that the insurance cover is due to expire. In most cases,
this notice will include an offer to renew the insurance for another year. The renewal notice will include important facts
about the policy, any changes to the policy terms and a price.

In a small number of cases, the insurer may not renew the policy.

We automatically renew most policies. This means that, unless you tell us otherwise, your new insurance cover will start
on your renewal date. If we intend to automatically renew the policy, we will tell you this in your renewal notice.

You must contact us at least 7 days before the insurance policy expires to (i) cancel the renewal or (ii) negotiate the
renewal payment.


# CANCELLATIONS

This section contains important information about your rights, plus our and your insurer's rights of cancellation.


## YOUR RIGHTS TO CANCEL THIS POLICY

You have the right to cancel this policy within the first 14 days without incurring a penalty and without giving a reason. This
is known as "the 14-day cooling off period" and starts on (i) the day this policy is entered into or (ii) the day on which you
receive these terms and conditions, whichever is later.


## POLICYHOLDER OF A CAR POLICY

Cancellation can only be authorised by the policyholder and it's your responsibility to notify any other drivers named on
the policy that they are no longer insured.


## WHAT HAPPENS WHEN THE POLICY IS CANCELLED

If the policy is cancelled, any fees, such as the arrangement fee, incurred before cancellation are non-refundable, as is the
cost of your insurance for the number of days you have been insured.

If the policy is cancelled, your insurer won't refund a premium for any car where a non-recoverable claim has been made
on the car or any replacement car during the period of cover. Where instalments are being paid under a credit agreement,
the balance of the annual premium and the cancellation fee (if it's 14 days or more since your policy started) will need to
be paid.

Following the cancellation, we will calculate the refund as follows: If the cancellation of the policy is before the cover starts:

. The person who paid the costs of insurance will be entitled to a full refund of the insurer premium minus our non-
refundable fees as shown on your cover summary document

If the cancellation of the policy is within the 14-day cooling off period:

. The person who paid the costs of insurance will be entitled to a premium refund on a pro-rata basis for the period
of cover that hasn't been used minus our non-refundable fees as shown on your cover summary document.

<!-- PageFooter="Insurance Policy | Page 13" -->


In [6]:
# Gets the parsed VehicleInsurancePolicy object from the completion response.
insurance_policy = completion.choices[0].message.parsed

# Determines the accuracy of the extracted data against the expected values.
accuracy = insurance_policy_evaluator.evaluate(insurance_policy)

# Determines the confidence of the extracted data using both the OpenAI and Azure Document Intelligence responses.
di_confidence = di_evaluate_confidence(insurance_policy.to_dict(), result)
oai_confidence = oai_evaluate_confidence(insurance_policy.to_dict(), completion.choices[0])

confidence = merge_confidence_values(di_confidence, oai_confidence)

# Gets the total execution time of the data extraction process.
total_elapsed = di_stopwatch.elapsed + image_stopwatch.elapsed + oai_stopwatch.elapsed

# Gets the prompt tokens and completion tokens from the completion response.
prompt_tokens = completion.usage.prompt_tokens
completion_tokens = completion.usage.completion_tokens

# Save the output of the data extraction result.
extraction_result = DataExtractionResult(insurance_policy.to_dict(), confidence, accuracy, prompt_tokens, completion_tokens, total_elapsed)

with open(f"{working_dir}/samples/extraction/vision-based/comprehensive.{pdf_file_name}.json", "w") as f:
    f.write(extraction_result.to_json(indent=4))
    
# Display the outputs of the data extraction process.
df = pd.DataFrame([
    {
        "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
        "Confidence": f"{confidence['_overall'] * 100:.2f}%",
        "Execution Time": f"{total_elapsed:.2f} seconds",
        "Document Intelligence Execution Time": f"{di_stopwatch.elapsed:.2f} seconds",
        "Image Pre-processing Execution Time": f"{image_stopwatch.elapsed:.2f} seconds",
        "OpenAI Execution Time": f"{oai_stopwatch.elapsed:.2f} seconds",
        "Prompt Tokens": prompt_tokens,
        "Completion Tokens": completion_tokens
    }
])

display(Markdown(df.to_markdown(index=False, tablefmt='unsafehtml')))
display(Markdown(extraction_comparison(expected.to_dict(), insurance_policy.to_dict(), confidence)))

<table>
<thead>
<tr><th>Accuracy  </th><th>Confidence  </th><th>Execution Time  </th><th>Document Intelligence Execution Time  </th><th>Image Pre-processing Execution Time  </th><th>OpenAI Execution Time  </th><th style="text-align: right;">  Prompt Tokens</th><th style="text-align: right;">  Completion Tokens</th></tr>
</thead>
<tbody>
<tr><td>100.00%   </td><td>99.56%      </td><td>29.52 seconds   </td><td>6.84 seconds                          </td><td>5.67 seconds                         </td><td>17.01 seconds          </td><td style="text-align: right;">          17655</td><td style="text-align: right;">                266</td></tr>
</tbody>
</table>

<table>
<thead>
<tr><th>Field                                          </th><th>Expected                                    </th><th>Extracted                                                                     </th><th style="text-align: right;">  Confidence</th></tr>
</thead>
<tbody>
<tr><td>policy_number                                  </td><td>GB20246717948                               </td><td><span style='color: green'>GB20246717948</span>                               </td><td style="text-align: right;">    1       </td></tr>
<tr><td>cost_annual_total                              </td><td>532.19                                      </td><td><span style='color: green'>532.19</span>                                      </td><td style="text-align: right;">    0.999994</td></tr>
<tr><td>cost_payable_by_date                           </td><td>2024-06-13                                  </td><td><span style='color: green'>2024-06-13</span>                                  </td><td style="text-align: right;">    0.999747</td></tr>
<tr><td>renewal_renewal_notification_date              </td><td>2025-05-12                                  </td><td><span style='color: green'>2025-05-12</span>                                  </td><td style="text-align: right;">    0.989057</td></tr>
<tr><td>renewal_last_date_to_renew                     </td><td>2025-05-26                                  </td><td><span style='color: green'>2025-05-26</span>                                  </td><td style="text-align: right;">    0.98511 </td></tr>
<tr><td>effective_from                                 </td><td>2024-06-03                                  </td><td><span style='color: green'>2024-06-03</span>                                  </td><td style="text-align: right;">    0.999999</td></tr>
<tr><td>effective_to                                   </td><td>2025-06-02                                  </td><td><span style='color: green'>2025-06-02</span>                                  </td><td style="text-align: right;">    0.999999</td></tr>
<tr><td>last_date_to_cancel                            </td><td>2024-06-17                                  </td><td><span style='color: green'>2024-06-17</span>                                  </td><td style="text-align: right;">    0.99126 </td></tr>
<tr><td>policyholder_first_name                        </td><td>Joe                                         </td><td><span style='color: green'>Joe</span>                                         </td><td style="text-align: right;">    0.999886</td></tr>
<tr><td>policyholder_last_name                         </td><td>Bloggs                                      </td><td><span style='color: green'>Bloggs</span>                                      </td><td style="text-align: right;">    1       </td></tr>
<tr><td>policyholder_date_of_birth                     </td><td>1990-01-05                                  </td><td><span style='color: green'>1990-01-05</span>                                  </td><td style="text-align: right;">    0.994408</td></tr>
<tr><td>policyholder_address                           </td><td>73 Regal Way, LEEDS, West Yorkshire, LS1 5AB</td><td><span style='color: green'>73 Regal Way, LEEDS, West Yorkshire, LS1 5AB</span></td><td style="text-align: right;">    0.998246</td></tr>
<tr><td>policyholder_email_address                     </td><td>Joe.Bloggs@me.com                           </td><td><span style='color: green'>Joe.Bloggs@me.com</span>                           </td><td style="text-align: right;">    0.999986</td></tr>
<tr><td>policyholder_total_years_of_residence_in_uk    </td><td>34                                          </td><td><span style='color: green'>34</span>                                          </td><td style="text-align: right;">    0.978635</td></tr>
<tr><td>policyholder_driving_license_number            </td><td>BLOGGS901050JJ1AB                           </td><td><span style='color: green'>BLOGGS901050JJ1AB</span>                           </td><td style="text-align: right;">    0.999996</td></tr>
<tr><td>vehicle_registration_number                    </td><td>VS24DMC                                     </td><td><span style='color: green'>VS24DMC</span>                                     </td><td style="text-align: right;">    1       </td></tr>
<tr><td>vehicle_make                                   </td><td>Hyundai                                     </td><td><span style='color: green'>Hyundai</span>                                     </td><td style="text-align: right;">    0.996648</td></tr>
<tr><td>vehicle_model                                  </td><td>IONIQ 5 Premium 73 kWh RWD                  </td><td><span style='color: green'>IONIQ 5 Premium 73 kWh RWD</span>                  </td><td style="text-align: right;">    0.999836</td></tr>
<tr><td>vehicle_year                                   </td><td>2024                                        </td><td><span style='color: green'>2024</span>                                        </td><td style="text-align: right;">    1       </td></tr>
<tr><td>vehicle_value                                  </td><td>40000.0                                     </td><td><span style='color: green'>40000.0</span>                                     </td><td style="text-align: right;">    0.955336</td></tr>
<tr><td>accident_excess_compulsory                     </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999998</td></tr>
<tr><td>accident_excess_voluntary                      </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    1       </td></tr>
<tr><td>accident_excess_unapproved_repair_penalty      </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.996636</td></tr>
<tr><td>fire_and_theft_excess_compulsory               </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    1       </td></tr>
<tr><td>fire_and_theft_excess_voluntary                </td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999999</td></tr>
<tr><td>fire_and_theft_excess_unapproved_repair_penalty</td><td>250                                         </td><td><span style='color: green'>250</span>                                         </td><td style="text-align: right;">    0.999948</td></tr>
</tbody>
</table>